In [14]:
import os
import math
import random
import pandas as pd
import wget

## File paths
atlasFile = 'atlas.csv'
listFile = 'listOfIds.txt'
webFile = 'websites.txt'
resultsFolder = 'results'
# initializers
ra, dec, radius = 0, 0, 0.0014
id = '0'
websites = []
atlasFile = pd.read_csv(atlasFile)
starter = 'https://irsa.ipac.caltech.edu/SIA?COLLECTION=spitzer_sha&RESPONSEFORMAT=CSV&FORMAT=image/csv&CALIB=2&POS=circle'

# Convert Panda series to arrays
objidA = pd.Series(atlasFile['objid']).array
raA = pd.Series(atlasFile['ra']).array
decA = pd.Series(atlasFile['dec']).array

atlasFile

,objid,ra,dec
0,SSTSL2 J152742.67+023551.8,231.927823,2.597747
1,SSTSL2 J053049.72-670554.9,82.707184,-67.098598
2,SSTSL2 J153607.88+384214.4,234.032847,38.704015
3,SSTSL2 J224656.98+602924.5,341.737456,60.490152
4,SSTSL2 J022523.15-733726.6,36.346464,-73.624064
...,...,...,...
17459,SSTSL2 J053328.89-683636.5,83.370390,-68.610158
17460,SSTSL2 J194613.16+263840.3,296.554874,26.644537
17461,SSTSL2 J193929.88+233823.8,294.874522,23.639946
17462,SSTSL2 J174449.11-333318.8,266.204645,-33.555244


In [17]:
# Asks for Object ID, then returns ra and dec of the object is available
# If not available, display error and ends program
# Edit: replaces spaces in ID with underscores
def search(id):
    for i in range(len(objidA)):
        if id == objidA[i]:
            ra = raA[i]
            dec = decA[i]
            id = id.replace(' ', '_')
            return (id, ra, dec)
    print(f'Error: search()\nInvalid id: {id}')
    exit()

# Rounds input to the 5th decimal and outputs string
def roundCords(num):
    temp = (num*100000)+0.5
    temp = math.floor(temp)
    temp = str(float(temp)/100000)
    return(temp)

# Outputs website
def makeWeb(r, d, rad):
    endP = f'{starter}+{r}+{d}+{rad}'
    return(endP)

# makes the website and adds to a list
def makeEntry(id):
    id, ra, dec = search(id)
    website = (makeWeb(roundCords(ra),roundCords(dec),roundCords(radius)))
    check = False

    #checks if the entry is already made
    for i in websites:
        if i[0] == id:
            check = True

    if check:
        print(f'{id} entry already in listOfIds.txt file')
    else:
        print(f'{id} entry made!')
        entry = [id, ra, dec, radius, website]
        websites.append(entry)
    return(websites)

# clears txt and writes all entries
def createTXT(websites):
    wordsInFile = ''
    for entry in websites:
        wordsInFile = wordsInFile + f'ID:{entry[0]}\nRA:{entry[1]} DEC:{entry[2]} RADIUS:{entry[3]}\n{entry[4]}\n\n'
    f = open(webFile,'w')
    f.write(wordsInFile)
    f.close
    print('Updated File!\n')

# takes ids in listOfIds.txt into a list
def listOut():
    list = []
    myfile = open(listFile, "rt")
    contents = myfile.read() + '\n'
    myfile.close()
    while len(contents) > 0:
        index = contents.find('\n')
        id = contents[0:index]
        list.append(id)
        contents = contents[index+1:]
    return(list)

#makes a number of random obj #s
def randomIds(randomN):
    for i in range(randomN):
        websites = makeEntry(objidA[random.randrange(0, len(objidA))])
    return(websites)

# Downloads links
def grab(web):
    id = web[0]
    link = web[4]

    # adds all file names in 'results' in a list
    results = []
    for file_path in os.listdir(resultsFolder):
        if os.path.isfile(os.path.join(resultsFolder, file_path)):
            results.append(file_path)

    #checks if the pdf is already downloaded
    if (id+'.csv') in results:
        print(f'{id}.csv already in results folder\n')
    else:
        print('Grabbing...')
        wget.download(link, f'{id}.csv')
        os.rename(f'{id}.csv', f'{resultsFolder}/{id}.csv')
        print(f'Downloaded: {id}.csv\n')

# Random links option
def randomLinks():
    num = input('How many links: ')
    if num.isdigit:
        websites = randomIds(math.floor(int(num)))
        createTXT(websites)
        grabbing = input('Download links?(y/n): ')
        if grabbing.lower() == 'y':
            for i in range(len(websites)):
                grab(websites[i])
            print('Downloaded!')
    else:
        print('Invalid')

# Provided List option
def nonRandomLinks():
    listOfIds = listOut()
    for obj in range(len(listOfIds)):
        websites = makeEntry(listOfIds[obj])
    createTXT(websites)
    grabbing = input('Download links?(y/n): ')
    if grabbing.lower == 'y':
        for i in len(websites[i]):
            grab(websites)


In [18]:
start = input('Random links?(y/n): ')
if start.lower() == 'y':
    randomLinks()
elif start.lower() == 'n':
    nonRandomLinks()
else:
    print('Invalid')

SSTSL2_J021839.65-054400.1 entry made!
Updated File!

